In [48]:
!pip install SyntheticControlMethods

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.tencent.com/pypi/simple
  Using cached https://mirrors.tencent.com/yun/pypi/packages/81/b9/c87ed26799a76c5627571b3b9ab6344727b01a5ffd7e8dd4062bd6428373/SyntheticControlMethods-1.1.17.tar.gz (26 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      Looking in indexes: https://mirrors.tencent.com/pypi/simple
      Ignoring numpy: markers 'python_version == "3.7" and platform_machine == "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.8" and platform_machine != "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.8

In [49]:
#Import packages
import pandas as pd
import numpy as np

from SyntheticControlMethods import Synth, DiffSynth

In [60]:
#Import German Reunification data from paper
#Can be found in /datasets folder in repo
data = pd.read_csv("german_reunification.csv")
data = data.drop(columns="code", axis=1)
data.head()


,country,year,gdp,infrate,trade,schooling,invest60,invest70,invest80,industry
0,USA,1960,2879,NaN,9.693181,43.799999,NaN,NaN,NaN,NaN
1,USA,1961,2929,1.075182,9.444655,NaN,NaN,NaN,NaN,NaN
2,USA,1962,3103,1.116071,9.429324,NaN,NaN,NaN,NaN,NaN
3,USA,1963,3227,1.214128,9.470706,NaN,NaN,NaN,NaN,NaN
4,USA,1964,3420,1.308615,9.725879,NaN,NaN,NaN,NaN,NaN


In [61]:
sc = Synth(data, "gdp", "country", "year", 1990, "West Germany", n_optim=100)

ValueError: `f0` passed has more than 1 dimension.

## 拆解处理版本

In [53]:
# 2. 合成控制法 (Synthetic Control)
# 导入所需的库
from scipy.optimize import minimize
import numpy as np
import pandas as pd

# 生成模拟数据（1个处理单元，5个控制单元，10年）
np.random.seed(42)
years = 10
n_control = 5

# 生成处理单元的数据
treated_data = np.random.normal(0, 1, years) + np.arange(years) * 0.5 + (np.arange(years) >= 5) * 3

# 生成控制单元的数据
control_data = [np.random.normal(0, 1, years) + np.arange(years) * 0.5 for _ in range(n_control)]

# 合并处理单元和控制单元的数据
data = {
    'year': np.tile(np.arange(years), n_control + 1),
    'unit': np.repeat(['treated'] + [f'control_{i}' for i in range(n_control)], years),
    'y': np.concatenate([treated_data] + control_data)
}

# 创建 DataFrame
df = pd.DataFrame(data)

In [58]:
df

,year,unit,y
0,0,treated,0.496714
1,1,treated,0.361736
2,2,treated,1.647689
3,3,treated,3.023030
4,4,treated,1.765847
5,5,treated,5.265863
6,6,treated,7.579213
7,7,treated,7.267435
8,8,treated,6.530526
9,9,treated,8.042560


In [52]:
# 定义损失函数（最小化合成控制与处理单元干预前的差异）
def loss(weights, X, y):
    return np.sum((X @ weights - y) ** 2)

# 预处理数据，提取干预前的数据
pre_period = df[df['year'] < 5]

# 提取控制单元在干预前的数据
X = pre_period[pre_period['unit'] != 'treated'].pivot(index='year', columns='unit', values='y').values.T

# 提取处理单元在干预前的数据
y = pre_period[pre_period['unit'] == 'treated']['y'].values

# 优化求解权重
# 初始权重，每个控制单元权重相等
result = minimize(loss, x0=np.ones(n_control)/n_control, args=(X, y), 
                  bounds=[(0, 1)]*n_control, constraints={'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
weights = result.x

# 计算合成控制结果
synth = (df[df['unit'] != 'treated'].pivot(index='year', columns='unit', values='y') * weights).sum(axis=1)

# 计算处理效应
effect = df[df['unit'] == 'treated'].set_index('year')['y'] - synth

# 打印干预后处理效应的平均值
print("处理效应:", effect.loc[5:].mean())

处理效应: 3.77910492812899
